## BLOQUE 0 — Infraestructura, Rutas Maestras y Entorno Hardcore (v5.4)
Lead Data Scientist: Pedro J. Lancheros (Peter Boat)
Proyecto: CRSN Slides (3J / VitaNova)
Versión: v5.4 (Resiliencia de Directorios)
Sello: 2026-01-29 [jue] 10:20

🎯 Propósito
Establecer la infraestructura de producción. Este bloque inicializa el entorno, monta Google Drive y valida la existencia de los activos maestros. Implementa una lógica de persistencia que asegura la creación del directorio de salida antes de cualquier operación de escritura.

📥 Entradas
- Excel Maestro: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx
- Logo Circular: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-circulo.png

📤 Salidas
- Variables globales EXCEL_PATH, LOGO_PATH y OUTPUT_DIR validadas.
- Directorio de salida físico creado en Google Drive.

🛡️ Checks de Seguridad (Anti-Alucinación)
- [ ] Validación de Existencia: El script verifica físicamente los archivos en Drive.
- [ ] Persistencia Atómica: Generación de RUN_ID único para evitar colisiones.
- [ ] Auto-Recuperación: El bloque puede ser ejecutado múltiples veces sin romper la sesión.

In [11]:
# -----------------------------------------------------------------------------
# Autor:   Pedro J. Lancheros (Peter Boat)
# Fecha:   2026-01-29
# Hora:    10:20
# Versión: v5.4
# Proyecto: CRSN Slides (3J / VitaNova)
# Propósito: BLOQUE 0 - Inicialización de Infraestructura y Rutas Maestras
# -----------------------------------------------------------------------------

import os
import sys
import subprocess
from datetime import datetime

def initialize_production_environment():
    """Inicializa entorno, monta Drive y valida rutas maestras."""
    print("📡 [SISTEMA] Desplegando Arquitectura Hardcore v5.4...")

    # 1. Instalación de dependencias
    libs = ["pyarrow", "python-pptx", "openpyxl", "pandas", "matplotlib", "fpdf2", "numpy"]
    for lib in libs:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", lib])

    import pandas as pd
    from google.colab import drive

    # 2. Montaje de Drive
    if not os.path.exists('/content/drive'):
        print("📡 [DRIVE] Solicitando acceso a la unidad...")
        drive.mount('/content/drive')

    # 3. RUTAS MAESTRAS DEFINIDAS
    EXCEL_PATH_FINAL = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx"
    LOGO_PATH_FINAL = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-circulo.png"

    # 4. GESTIÓN DE SALIDAS (Trazabilidad)
    TIMESTAMP = datetime.now().strftime("%Y-%m-%d_%H%M")
    RUN_ID = f"CRSN_PROD_v5.4_{TIMESTAMP}"
    BASE_OUT = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025"
    OUTPUT_DIR_FINAL = os.path.join(BASE_OUT, RUN_ID)

    # CREACIÓN FÍSICA DEL DIRECTORIO
    os.makedirs(OUTPUT_DIR_FINAL, exist_ok=True)
    print(f"✅ [SISTEMA] Directorio de salida verificado: {OUTPUT_DIR_FINAL}")

    # 5. VERIFICACIÓN DE ACTIVOS
    activos = {"Excel": EXCEL_PATH_FINAL, "Logo": LOGO_PATH_FINAL}
    for nombre, path in activos.items():
        if os.path.exists(path):
            print(f"✅ [EXISTE] {nombre} en: {path}")
        else:
            print(f"❌ [ERROR] {nombre} NO hallado en {path}")
            raise FileNotFoundError(f"Falta activo crítico: {path}")

    return EXCEL_PATH_FINAL, LOGO_PATH_FINAL, OUTPUT_DIR_FINAL

# EJECUCIÓN GLOBAL
try:
    EXCEL_PATH, LOGO_PATH, OUTPUT_DIR = initialize_production_environment()
    print(f"\n✅ [BLOQUE 0] FINALIZADO. Listo para procesamiento.")
except Exception as e:
    print(f"❌ [CRÍTICO] Fallo en la inicialización: {e}")

📡 [SISTEMA] Desplegando Arquitectura Hardcore v5.4...
✅ [SISTEMA] Directorio de salida verificado: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/CRSN_PROD_v5.4_2026-01-29_1519
✅ [EXISTE] Excel en: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx
✅ [EXISTE] Logo en: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-circulo.png

✅ [BLOQUE 0] FINALIZADO. Listo para procesamiento.


## BLOQUE 1 — Ingesta Quirúrgica y Validación de Diezmos (v5.4)
Lead Data Scientist: Pedro J. Lancheros (Peter Boat)
Proyecto: CRSN Slides (3J / VitaNova)
Versión: v5.4 (Rigor de Auditoría)
Sello: 2026-01-29 [jue] 10:22

🎯 Propósito
Consolidar la información financiera de Diezmos utilizando el CSV auditado. Este bloque asegura la persistencia en el directorio validado y calcula el Índice de Crecimiento (Base 2015=100) junto con la variación YoY para el reporte ejecutivo.

📥 Entradas
- CSV Auditado: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/Copy of diezmos_auditados_3J_ISO_v2.1.csv

📤 Salidas
- df_diezmos_master.parquet: Dataset consolidado guardado en Drive.
- df_diezmos_publico: DataFrame en memoria para renderizado de slides.

🛡️ Checks de Seguridad (Anti-Alucinación)
- [ ] Validación de Directorio: Check previo de existencia de OUTPUT_DIR antes de guardar.
- [ ] Normalización de Índice: El valor de 2015 es el denominador estricto.
- [ ] Integridad de Datos: Se eliminan registros fuera del rango 2015-2025.

In [12]:
# -----------------------------------------------------------------------------
# Autor:   Pedro J. Lancheros (Peter Boat)
# Fecha:   2026-01-29
# Hora:    10:22
# Versión: v5.4
# Proyecto: CRSN Slides (3J / VitaNova)
# Propósito: BLOQUE 1 - Consolidación de Diezmos Auditados
# -----------------------------------------------------------------------------

import pandas as pd
import numpy as np
import os
import gc

def procesar_diezmos_auditados(path_csv, output_dir):
    """Procesa la data financiera con validación de persistencia."""

    # 1. Check de Seguridad: Directorio de Salida
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
        print(f"⚠️ [SISTEMA] Directorio recuperado: {output_dir}")

    print(f"📡 [FINANZAS] Procesando: {os.path.basename(path_csv)}")

    # 2. Carga y Agregación
    df_raw = pd.read_csv(path_csv)
    df_raw.columns = [c.upper().replace(' ', '_') for c in df_raw.columns]

    # Definir columnas de valor (COP) y tiempo (AÑO)
    col_valor = 'COP' if 'COP' in df_raw.columns else 'VALOR'
    df_anual = df_raw.groupby('AÑO')[col_valor].sum().reset_index()
    df_anual = df_anual.rename(columns={col_valor: 'VALOR_REAL'})

    # Rango CRSN 2015-2025
    df_anual = df_anual[(df_anual['AÑO'] >= 2015) & (df_anual['AÑO'] <= 2025)].copy()

    # 3. Cálculo de Índices
    if 2015 not in df_anual['AÑO'].values:
        raise ValueError("Error: Falta año base 2015 en la data.")

    val_base = df_anual.loc[df_anual['AÑO'] == 2015, 'VALOR_REAL'].values[0]
    df_anual['INDICE'] = (df_anual['VALOR_REAL'] / val_base * 100).round(0).astype(int)
    df_anual['VAR_YOY'] = (df_anual['VALOR_REAL'].pct_change() * 100).fillna(0).round(0).astype(int)

    return df_anual

# EJECUCIÓN
try:
    PATH_CSV = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/Copy of diezmos_auditados_3J_ISO_v2.1.csv"

    # Uso de variables globales del Bloque 0
    df_diezmos_master = procesar_diezmos_auditados(PATH_CSV, OUTPUT_DIR)

    # Guardado en Parquet
    parquet_path = os.path.join(OUTPUT_DIR, "df_diezmos_master.parquet")
    df_diezmos_master.to_parquet(parquet_path, engine='pyarrow', index=False)

    print(f"\n✅ [BLOQUE 1] FINALIZADO.")
    print(f"📁 Datos persistidos en: {parquet_path}")
    print(df_diezmos_master[['AÑO', 'INDICE', 'VAR_YOY']].to_string(index=False))

except Exception as e:
    print(f"❌ [CRÍTICO] Error en Bloque 1: {e}")

finally:
    gc.collect()

📡 [FINANZAS] Procesando: Copy of diezmos_auditados_3J_ISO_v2.1.csv

✅ [BLOQUE 1] FINALIZADO.
📁 Datos persistidos en: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/CRSN_PROD_v5.4_2026-01-29_1519/df_diezmos_master.parquet
 AÑO  INDICE  VAR_YOY
2015     100        0
2016     124       24
2017     143       15
2018     162       13
2019     174        7
2020     177        2
2021     224       27
2022     194      -13
2023     221       14
2024     265       20
2025     271        2


## BLOQUE 3 — Renderizado DARK 16:9 con Blindaje de Márgenes (v6.2)
Lead Data Scientist: Pedro J. Lancheros (Peter Boat)
Proyecto: CRSN Slides (3J / VitaNova)
Versión: v6.2 (Control de Márgenes y Maximización de Fuente)
Sello: 2026-01-29 [jue] 11:15

🎯 Propósito
Corregir el recorte visual de los años y maximizar la legibilidad. Se establece un margen de seguridad del 5% en todos los bordes. Se amplía el tamaño de los años (eje X) y de las etiquetas de datos para que coincidan con el ancho de las columnas.

📥 Entradas
- df_diezmos_master: Serie histórica 2015-2025.
- LOGO_DARK: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-dark-circulo.png

📤 Salidas
- [PJLA_TIMESTAMP]_DIEZMOS_DARK_v6.2_FINAL.pptx

🛡️ Checks de Seguridad (Anti-Alucinación)
- [ ] Safe Zone Check: Margen de 0.5" en los 4 costados del slide.
- [ ] Escala de Texto: Años y datos en Bold con tamaño optimizado para el ancho de barra.
- [ ] Sin Recortes: Uso de plt.subplots_adjust para evitar el corte de labels inferiores.

In [20]:
# -----------------------------------------------------------------------------
# Autor:   Pedro J. Lancheros (Peter Boat)
# Fecha:   2026-01-29
# Hora:    11:15
# Versión: v6.2
# Proyecto: CRSN Slides (3J / VitaNova)
# Propósito: BLOQUE 3 - Slide DARK 16:9 Sin Recortes y Máximo Resaltado
# -----------------------------------------------------------------------------

import os
import pandas as pd
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from datetime import datetime

# CONFIGURACIÓN DE RUTAS MAESTRAS
OUTPUT_FOLDER = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/"
LOGO_DARK = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-dark-circulo.png"

def generar_pptx_perfecto_v62(df, logo_path, out_dir):
    """Genera slide 16:9 con márgenes de seguridad y fuentes maximizadas."""

    # 1. NOMENCLATURA PJLA
    now = datetime.now()
    prefix = now.strftime("%Y%m%d_%H%M")
    filename = f"{prefix}_DIEZMOS_DARK_v6.2_FINAL.pptx"
    full_save_path = os.path.join(out_dir, filename)

    # 2. SETUP SLIDE 16:9 CON MÁRGENES (5%)
    prs = Presentation()
    prs.slide_width, prs.slide_height = Inches(13.33), Inches(7.5)
    slide = prs.slides.add_slide(prs.slide_layouts[6])

    # Fondo Dark Profundo
    background = slide.background
    background.fill.solid()
    background.fill.fore_color.rgb = RGBColor(10, 10, 20)

    # 3. LOGO DARK (Escala Áurea)
    if os.path.exists(logo_path):
        slide.shapes.add_picture(logo_path, Inches(0.6), Inches(0.5), height=Inches(1.2))

    # 4. TÍTULOS (Subtítulo 17% y pegado)
    t_size = 60
    s_size = int(t_size * 0.17)

    t_box = slide.shapes.add_textbox(Inches(2.4), Inches(0.4), Inches(10), Inches(1))
    p = t_box.text_frame.paragraphs[0]
    p.text = "DIEZMOS 2015-2025"
    p.font.bold, p.font.size, p.font.color.rgb = True, Pt(t_size), RGBColor(255, 255, 255)

    s_box = slide.shapes.add_textbox(Inches(2.4), Inches(1.3), Inches(4), Inches(0.5))
    sp = s_box.text_frame.paragraphs[0]
    sp.text = "2015=100"
    sp.font.size, sp.font.color.rgb = Pt(s_size), RGBColor(255, 255, 255)

    # 5. GRÁFICO MAXIMIZADO (Sin recortes)
    # Aumentamos DPI para nitidez de texto
    plt.figure(figsize=(14, 6), facecolor='#0A0A14')
    ax = plt.axes()
    ax.set_facecolor('#0A0A14')

    # width=0.95 para cubrir el ancho de la columna
    bars = plt.bar(df['AÑO'].astype(str), df['INDICE'], color='#1A73E8', alpha=0.9, width=0.95)

    # Ajuste de Ejes para evitar recortes (Margen 5%)
    plt.xticks(color='white', fontsize=16, fontweight='black') # Años gigantes y resaltados
    plt.yticks(color='white', fontsize=12)
    ax.spines['bottom'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.grid(axis='y', color='white', linestyle='--', alpha=0.05)

    # Etiquetas de Datos (Data Labels) maximizadas al ancho de columna
    for bar in bars:
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, f'{int(bar.get_height())}',
                 ha='center', va='bottom', color='white', fontweight='black', fontsize=18)

    # AJUSTE CRÍTICO: Previene el recorte de los años abajo
    plt.tight_layout(pad=3.0)

    temp_img = "final_slide_v62.png"
    plt.savefig(temp_img, dpi=300, bbox_inches='tight', facecolor='#0A0A14')
    plt.close()

    # Inserción Central con margen de seguridad del 5%
    # Slide de 13.33", imagen de 12" centrada
    slide.shapes.add_picture(temp_img, Inches(0.65), Inches(2.1), width=Inches(12))

    # 6. GUARDADO
    if not os.path.exists(out_dir): os.makedirs(out_dir)
    prs.save(full_save_path)
    if os.path.exists(temp_img): os.remove(temp_img)
    return full_save_path

# EJECUCIÓN
try:
    final_pptx = generar_pptx_perfecto_v62(df_diezmos_master, LOGO_DARK, OUTPUT_FOLDER)
    print(f"✅ [ÉXITO] Entregable v6.2 generado y blindado: {final_pptx}")
    from google.colab import files
    files.download(final_pptx)
except Exception as e:
    print(f"❌ [ERROR] Fallo en Bloque 3: {e}")

✅ [ÉXITO] Entregable v6.2 generado y blindado: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/20260129_1605_DIEZMOS_DARK_v6.2_FINAL.pptx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## ESTÁNDARES DE PRODUCCIÓN — PROTOCOLO PJLA v6.2
Lead Data Scientist: Pedro J. Lancheros (Peter Boat)
Proyecto: CRSN Slides (3J / VitaNova)

🎯 Reglas de Oro para Slides:
1. Formato: Siempre Panorámico 16:9 (13.33" x 7.5").
2. Estética: Dark Mode (Fondo Navy #0A0A14, Barras Azul Neón #1A73E8).
3. Márgenes: Safe Zone del 5% en todos los bordes (Sin recortes).
4. Barras: Ancho del 95% de la columna (Efecto bloque sólido).
5. Tipografía: Años y Data Labels en 'Black' (Extra Bold) maximizados al ancho de la barra.
6. Títulos: Principal (100% size) / Subtítulo (17% size) pegado inmediatamente abajo.
7. Logo: Versión Dark, Proporción Áurea, margen superior izquierdo.
8. Nomenclatura: [YYYYMMDD_HHMM]_Nombre_Slide_v6.2.pptx

In [21]:
# -----------------------------------------------------------------------------
# Protocolo de Sincronización GitHub - PJLA v1.1
# -----------------------------------------------------------------------------

import os

# CONFIGURACIÓN (Usa tus datos registrados)
GITHUB_USER = "tu_usuario"
GITHUB_TOKEN = "tu_token"
REPO_NAME = "crsn-slides-v5-hardcore"

# 1. Copiar activos desde Drive/Local al entorno de Git
# Aseguramos que el logo dark esté incluido
os.system("cp /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-dark-circulo.png .")
os.system("cp /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-circulo.png .")

# 2. Protocolo de Actualización
os.system("git add .")
os.system('git commit -m "Update v6.2: Implementación Gold Standard, Logo Dark y Formato 16:9"')

# 3. Empuje a Main (Push)
os.system("git push origin main")

print(f"🚀 [GITHUB] Repositorio actualizado con la versión v6.2")

🚀 [GITHUB] Repositorio actualizado con la versión v6.2
